In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.read_csv('EX2_PPSWR.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

C:\Users\node1\AppData\Local\Temp\ipykernel_11128\819888444.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('EX2_PPSWR.csv')


Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU'],
      dtype='object')

In [3]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example2-PPSWR.RSE', "r") as f:
    print(type(f))
    for line in f:
        print(line)
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

<class '_io.TextIOWrapper'>
'Example2.RSE â€“ file will be as follows:

#SAMPLING PPSWR-SSS	'PPSWR-SSS Key word will be used for this scheme

#GROUP 2			'Number of Group variables used

SEC   C sec 	'Sector Rural-1, Urban-2

AGE_GR C NEWVAR 	'Age Groups



#FILTER 0		'Number of conditions used to generate the Work-file



#VARIABLE 4  	'Number of Variables selected

YP	N  NEWVAR	'Variable for Population

YM	N  NEWVAR	'Variable for Male

YF	N  NEWVAR	'Variable for Female

YH	N  NEWVAR	'Variable for Higher Secondary educated persons



#RENAME  5    			'Number of variables to be renamed

STRMID C  sec+st+strm+sstrm	'Combination of fields to get unique Stratum ID

FSU    C  fsu			' FSU code

SSS    C  sss			'Second Stage Stratum (SSS) Code

USU    C  ssu			'Ultimate stage unit, here Households

NFSU   C  nfsu			'No. of FSU surveyed in STRMID

NSS    N  nss			'No. of FSU surveyed in STRMID X SSS

MULT   N  mult			'Multiplier X 100



#TRANSFORM 5  	' Number of Variable transformed

AGE_GR=

In [4]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

In [5]:
### ---------------------------------------------------------------------------------------------------------
def GROUP_(data, df):
    for key, value in data.items():
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        print(key, value)
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                print(lst[0], lst[1])

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

### ---------------------------------------------------------------------------------------------------------
def FILTER_(data, df):
    lst_queries = []
    for key, value in data.items():

        if str('FILTER').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\([^()]+\)|\w+[<>!=]+[0-9]+(?:\s*(?:&&|\|\|)\s*(?:\([^()]+\)|\w+[<>!=]+[0-9]+))*'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst_queries.append(match.group())

        else:
            pass

    filter_query = ' & '.join(['({})'.format(item) for item in lst_queries])

    try:
        df = df.query(filter_query)
        return df
    except:
        return df


### ---------------------------------------------------------------------------------------------------------
def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

### ---------------------------------------------------------------------------------------------------------
def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()

### ---------------------------------------------------------------------------------------------------------
def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]

            for name in value:
                df[name] = 1

### ---------------------------------------------------------------------------------------------------------
def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name

                var_value = line_values[1].strip().split()[0]  # extract the variable value


                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)

                    if match:
                        condition = match.group(1)

                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3

### ---------------------------------------------------------------------------------------------------------
def EST_RSE_(data):
    for key, value in data.items():
        pass

df = FILTER_(data, df)
GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)



['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']


In [6]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")

    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass

    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")

    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=').\
                                        replace('NOT', '~')

    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'\1==', query_string)


    # # Define a function to replace variable names with df["<variable_name>"]
    # def replace_var(match):
    #     return f'df["{match.group(0)}"]'

    # # Use regex to replace variable names with df["<variable_name>"]
    # pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    # new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)

    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

### Calling the final_parsed_query function
### ---------------------------------------
final_parsed_query

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']


['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 None,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [7]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
['AGE_GR', '("01") AGE<5']
['AGE_GR', '("02") AGE>', '5 && AGE<10']
['AGE_GR', '("03") AGE>', '10 && AGE<20']
['AGE_GR', '("04") AGE>', '20 && AGE<60']
['AGE_GR', '("05") AGE>', '60']
['YP', '(1)']
['YM', '(1)        SEX<>2']
['YM', '(0)\t\tOtherwise']
['YF', '(1)        SEX', '2']
['YF', '(0)\t\tOtherwise']
['YH', '(1)        GEDU>', '"11"']
['YH', '(0)\t\tOtherwise']
[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]


In [8]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [9]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['AGE<5',
 'AGE>=5 & AGE<10',
 'AGE>=10 & AGE<20',
 'AGE>=20 & AGE<60',
 'AGE>=60',
 True,
 'SEX!=2',
 'Otherwise',
 'SEX==2',
 'Otherwise',
 'GEDU>=11',
 'Otherwise']

In [10]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set(['~(' + Arr3[j] + ')' for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = ' & '.join(other_values)
        print(new_value)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)
        print(value)

# new_arr3

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
True
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [11]:
Arr3 = new_arr3

In [12]:
for i in range(len(Arr2)):
    if type(Arr3[i]) == str:
        print(Arr3[i])

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [13]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):

    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]

    else:
        # try:
        # query_str = re.sub(r'0?\s?(\d+)', r'\1', query_str)
        print(query_str)
        df_query = df.query(query_str)
        # except:
        #     pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

AGE<5
AGE>=5 & AGE<10
AGE>=10 & AGE<20
AGE>=20 & AGE<60
AGE>=60
SEX!=2
~(SEX!=2)
SEX==2
~(SEX==2)
GEDU>=11
~(GEDU>=11)


In [14]:
# df = pd.read_csv('WF_PPSWR.csv')
# df.head()

In [15]:
# Remove leading and trailing spaces from column names
# df.columns = df.columns.str.strip()

In [16]:
# new_col_names = {'sec':'SEC', 'age':'AGE', 'age_gr':'AGE_GR', 'strmid':'STRMID',
#                  'fsu':'FSU', 'sss':'SSS', 'capzs':'CAPZS', 'smlzs':'SMLZS', 'nfsu':'NFSU',
#                  'nss':'NSS', 'mult':'MULT', 'sex':'SEX', 'yp':'YP', 'ym':'YM', 'yf':'YF',
#                  'gedu':'GEDU', 'yh':'YH', 'cmult':'CMULT'}

# df.rename(columns=new_col_names, inplace=True)

In [17]:
df['MULT'] = pd.to_numeric(df['MULT'])

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df['MULT'] / 100


In [18]:
df.columns

Index(['SEC', 'ST', 'AST', 'DC', 'NSSR', 'STRM', 'SSTRM', 'FSU', 'SSS', 'SSU',
       'CAPZS', 'SMLZS', 'DSI', 'NFSU', 'NSS', 'MULT', 'CAPH', 'SMLH', 'SRL',
       'HHSIZE', 'RELIGION', 'SG', 'MPCE_ALL', 'MPCE_ST', 'REL', 'SEX', 'AGE',
       'MARST', 'GEDU', 'TEDU', 'AGE_GR', 'STRMID', 'YP', 'YM', 'YF', 'YH',
       'SS_MULT', 'CMULT'],
      dtype='object')

In [19]:

df = df[['SEC', 'AGE', 'AGE_GR', 'STRMID', 'FSU', 'SSS', 'SSU', 'CAPZS', 'SMLZS', 'NFSU', 'NSS', 'MULT', 'SEX', 'YP', 'YM', 'YF', 'GEDU', 'YH', 'CMULT']]
df.to_csv('WorkFile_PPSWR_Method_Example1.csv')

In [20]:
# df = pd.read_excel('new_wf_ppswr_122532.xlsx')
# df = df.rename(columns=lambda x: x.strip())

In [21]:
bin(8)

'0b1000'

In [22]:
def generate_binary_combinations(n):
    binary_combinations = []
    for i in range(2 ** n):
        binary_combination = bin(i)[2:]
        binary_combination = '0' * (n - len(binary_combination)) + binary_combination
        binary_combinations.append(binary_combination)
    return binary_combinations

# limit = int(input("Enter - "))
limit = 2
binary_of_no = generate_binary_combinations(limit)
print(binary_of_no)

['00', '01', '10', '11']


In [23]:
def generate_variable_mapping(GROUP, binary_combinations):
    mapping = {}

    for comb in binary_combinations:
        variable_values = []
        consecutive_zeros = 0

        for i, bit in enumerate(comb):
            if bit == '0':
                consecutive_zeros += 1
                variable_values.append('Z' * consecutive_zeros)


            else:
                consecutive_zeros = 0  # Reset the consecutive_zeros count
                variable_values.append(GROUP[i])
        mapping[comb] = variable_values

    return mapping

###
GROUP = GROUP_(data, df)

# Create an empty DataFrame
df_new = pd.DataFrame()

n = len(GROUP)
binary_combinations = generate_binary_combinations(n)
variable_mapping = generate_variable_mapping(GROUP, binary_combinations)

VALUES = []
print("Customized Mapping:")
for comb, values in variable_mapping.items():
    print(f"Binary Combination '{comb}': {values}")
    VALUES.append(values)

Customized Mapping:
Binary Combination '00': ['Z', 'ZZ']
Binary Combination '01': ['Z', 'AGE_GR']
Binary Combination '10': ['SEC', 'Z']
Binary Combination '11': ['SEC', 'AGE_GR']


In [24]:
### Storing group variables into temporary dataframe
temp_df = df[GROUP].copy()

# Create a dictionary to store the updates
updates = {}

Tables = []

W1 = []
W2 = []
W3 = []
W4 = []
W5 = []
W6 = []
W7 = []


### Iterate through GROUP and VALUES
for value in range(len(VALUES)):

    df_org = df.copy()

    for i in range(len(GROUP)):
        if GROUP[i] != VALUES[value][i]:
            updates[GROUP[i]] = VALUES[value][i]

        if GROUP[i] == VALUES[value][i]:
            updates[GROUP[i]] = temp_df[GROUP[i]]

    # Update the DataFrame with the specified values
    df_org.update(pd.DataFrame(updates, index=df.index))

    # print(df)

    # GROUP = ['SEC', 'ST_GR']
    new_groupby_cols_subset = ['STRMID']

    # Step 1: Generate Group level estimates Yhat & Xhat
    df_org['YP'] = pd.to_numeric(df_org['YP']) * df_org['CMULT']
    df_org['YM'] = pd.to_numeric(df_org['YM']) * df_org['CMULT']
    df_org['YF'] = pd.to_numeric(df_org['YF']) * df_org['CMULT']
    df_org['YH'] = pd.to_numeric(df_org['YH']) * df_org['CMULT']

    w1 = df_org.groupby(GROUP).agg(
                                            no_sam=('YP', 'size'),
                                             yphat=('YP', 'sum'),
                                              ymhat=('YM', 'sum'),
                                               yfhat=('YF', 'sum'),
                                                yhhat=('YH', 'sum')
                                            ).reset_index()


    # Step 2: Generate STRMID (X SSS) level Estimates of each variable for the same combination:
    df_org['FSU'] = df_org['FSU'].astype(str)
    df_org['SSS'] = df_org['SSS'].astype(str)

    df_org['UN_FSU'] = df_org['FSU'] + df_org['SSS']


    w2 = df_org.groupby(GROUP + ['STRMID', 'SSS']).agg(
                                                             nd_fsu = ('UN_FSU', 'nunique'),
                                                              ypstr=('YP', 'sum'),
                                                               ymstr=('YM', 'sum'),
                                                                yfstr=('YF', 'sum'),
                                                                 yhstr=('YH', 'sum')
                                                            ).reset_index()


    # Step 3: Generate STRMID (X SSS) X FSU level Estimates of each variable for the same combination: Say Yfsu & Xfsu
    df_org['NS'] = df_org['NSS']
    w3 = df_org.groupby(GROUP + ['STRMID', 'SSS', 'FSU', 'NS']).agg(
                                                                          ypfsu=('YP', 'sum'),
                                                                           ymfsu=('YM', 'sum'),
                                                                            yffsu=('YF', 'sum'),
                                                                             yhfsu=('YH', 'sum')
                                                                        ).reset_index()



    # Step 4: Combine all three estimates at STRMID (X SSS) level for the same combination
    w4 = pd.merge(w3, w2, on=['SEC', 'AGE_GR', 'STRMID', 'SSS'], suffixes=('', '_w2'))

    # Combining YHAT & XHAT
    w1['r1hat'] = w1['yfhat'] / w1['ymhat']
    w1['r2hat'] = w1['yhhat'] / w1['yphat']

    w5 = pd.merge(w4, w1[['SEC', 'AGE_GR', 'yphat', 'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat']], on=['SEC', 'AGE_GR'])

    W1.append(w1)
    W2.append(w2)
    W3.append(w3)
    W4.append(w4)
    W5.append(w5)

In [25]:
for i in range(len(W5)):
    print(W5[i].columns)

Index(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS', 'ypfsu', 'ymfsu',
       'yffsu', 'yhfsu', 'nd_fsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr', 'yphat',
       'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS', 'ypfsu', 'ymfsu',
       'yffsu', 'yhfsu', 'nd_fsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr', 'yphat',
       'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS', 'ypfsu', 'ymfsu',
       'yffsu', 'yhfsu', 'nd_fsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr', 'yphat',
       'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat'],
      dtype='object')
Index(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS', 'ypfsu', 'ymfsu',
       'yffsu', 'yhfsu', 'nd_fsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr', 'yphat',
       'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat'],
      dtype='object')


In [26]:
# from numba import jit

# for i in range(len(W5)):

#     @jit(nopython=True)
#     def calculate_myfym_w6_np(NS, nd_fsu, yfstr, r1hat, ymstr):
#         return np.sum(((NS * (yffsu - r1hat * ymfsu)) - (yfstr - r1hat * ymstr)) ** 2)

#     def calculate_myfym_w6(x):
#         NS = x['NS'].values
#         yffsu = x['yffsu'].values
#         r1hat = x['r1hat'].values
#         ymfsu = x['ymfsu'].values
#         yfstr = x['yfstr'].values
#         ymstr = x['ymstr'].values

#         return calculate_myfym_w6_np(NS, nd_fsu, yfstr, r1hat, ymstr)
    
    
#     @jit(nopython=True)
#     def calculate_vyh_w6_np(NS, nd_fsu, yhstr):
#         return np.sum((NS * yhfsu - yhstr) ** 2)
    
#     def calculate_vyh_w6(x):
#         NS = x['NS'].values
#         yhfsu = x['yhfsu'].values
#         yhstr = x['yhstr'].values

#         return calculate_vyh_w6_np(NS, nd_fsu, yhstr)
    

#     @jit(nopython=True)
#     def calculate_vyh_w6_np(NS, nd_fsu, yhstr, r1hat, ypstr):
#         return np.sum(((NS * (yhfsu - r2hat * ypfsu)) - (yhstr - r2hat * ypstr)) ** 2)
    
#     def calculate_myhyp_w6(x):
#         NS = x['NS'].values
#         yhfsu = x['yhfsu'].values
#         r2hat = x['r2hat'].values
#         ypfsu = x['ypfsu'].values
#         yhstr = x['yhstr'].values
#         ypstr = x['ypstr'].values

#         return calculate_myhyp_w6_np(NS, nd_fsu, yhstr, r2hat, ypstr)

#     w6 = W5[i].groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
#                     lambda x: pd.Series({
#                         'myfym': calculate_myfym_w6(x),
#                         'vyh': calculate_vyh_w6(x),
#                         'myhyp': calculate_myhyp_w6(x)
#                     })
#                 ).reset_index()

#     w6A = w6.copy()

#     W6.append(w6)


#     print(w6)


###################################################
############ More optimised code ##################
###################################################

from numba import jit

# Decorate your functions with @jit
@jit(nopython=True)
def calculate_myfym_np(NS, yffsu, r1hat, ymfsu, yfstr, ymstr):
    return np.sum(((NS * (yffsu - r1hat * ymfsu)) - (yfstr - r1hat * ymstr)) ** 2)

@jit(nopython=True)
def calculate_vyh_np(NS, yhfsu, yhstr):
    return np.sum((NS * yhfsu - yhstr) ** 2)

@jit(nopython=True)
def calculate_myhyp_np(NS, yhfsu, r2hat, ypfsu, yhstr, ypstr):
    return np.sum(((NS * (yhfsu - r2hat * ypfsu)) - (yhstr - r2hat * ypstr)) ** 2)

W6 = []

for i in range(len(W5)):
    w6 = W5[i].groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
        lambda x: pd.Series({
            'myfym': calculate_myfym_np(x['NS'].values, x['yffsu'].values, x['r1hat'].values, x['ymfsu'].values, x['yfstr'].values, x['ymstr'].values),
            'vyh': calculate_vyh_np(x['NS'].values, x['yhfsu'].values, x['yhstr'].values),
            'myhyp': calculate_myhyp_np(x['NS'].values, x['yhfsu'].values, x['r2hat'].values, x['ypfsu'].values, x['yhstr'].values, x['ypstr'].values)
        })
    ).reset_index()

    W6.append(w6)
    print(w6)

      SEC AGE_GR    STRMID SSS   NS  nd_fsu         myfym           vyh  \
0       Z     ZZ  01010101   1  2.0       2  6.795921e+04  0.000000e+00   
1       Z     ZZ  01010101   2  2.0       1  2.610635e+03  2.047189e+04   
2       Z     ZZ  01010101   3  2.0       2  1.702239e+05  0.000000e+00   
3       Z     ZZ  01010101   4  2.0       2  2.949352e+03  0.000000e+00   
4       Z     ZZ  01010101   6  2.0       1  3.450790e+03  1.279493e+03   
...    ..    ...       ...  ..  ...     ...           ...           ...   
25723   Z     ZZ  02363222   7  2.0       2  5.864874e+09  1.412410e+10   
25724   Z     ZZ  02363223   2  2.0       1  2.858902e+06  7.236100e+04   
25725   Z     ZZ  02363223   4  2.0       1  2.991649e+04  6.512490e+05   
25726   Z     ZZ  02363223   6  2.0       1  1.657597e+06  8.755681e+06   
25727   Z     ZZ  02363223   7  2.0       2  4.425689e+09  1.210325e+11   

              myhyp  
0      2.726342e+04  
1      3.482372e+03  
2      1.210301e+04  
3      4.84

In [27]:
W5[1].columns

Index(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'FSU', 'NS', 'ypfsu', 'ymfsu',
       'yffsu', 'yhfsu', 'nd_fsu', 'ypstr', 'ymstr', 'yfstr', 'yhstr', 'yphat',
       'ymhat', 'yfhat', 'yhhat', 'r1hat', 'r2hat'],
      dtype='object')

In [28]:
from numba import jit

for i in range(len(W5)):
    # Step 6: Add up to Group combinations and use the Estimate & RSE formulae to calculate final figures for that Group combination and store in a temporary file say T1.

    @jit(nopython=True)
    def calculate_myfym_w7_np(NS, nd_fsu, yfstr, r1hat, ymstr):
        return np.sum(((NS - nd_fsu) * (0 - (yfstr - r1hat * ymstr))) ** 2)

    def calculate_myfym_w7(x):
        NS = x['NS'].values
        nd_fsu = x['nd_fsu'].values
        yfstr = x['yfstr'].values
        r1hat = x['r1hat'].values
        ymstr = x['ymstr'].values

        return calculate_myfym_w7_np(NS, nd_fsu, yfstr, r1hat, ymstr)

    @jit(nopython=True)
    def calculate_vyh_w7_np(NS, nd_fsu, yhstr):
        return np.sum(((NS - nd_fsu) * (0 - yhstr)) ** 2)

    def calculate_vyh_w7(x):
        NS = x['NS'].values
        nd_fsu = x['nd_fsu'].values
        yhstr = x['yhstr'].values

        return calculate_vyh_w7_np(NS, nd_fsu, yhstr)

    @jit(nopython=True)
    def calculate_myhyp_w7_np(NS, nd_fsu, yhstr, r2hat, ypstr):
        return np.sum(((NS - nd_fsu) * (0 - (yhstr - r2hat * ypstr))) ** 2)

    def calculate_myhyp_w7(x):
        NS = x['NS'].values
        nd_fsu = x['nd_fsu'].values
        yhstr = x['yhstr'].values
        r2hat = x['r2hat'].values
        ypstr = x['ypstr'].values

        return calculate_myhyp_w7_np(NS, nd_fsu, yhstr, r2hat, ypstr)

    w7 = W5[i].groupby(['SEC', 'AGE_GR', 'STRMID', 'SSS', 'NS', 'nd_fsu']).apply(
        lambda x: pd.Series({
            'myfym': calculate_myfym_w7(x),
            'vyh': calculate_vyh_w7(x),
            'myhyp': calculate_myhyp_w7(x)
        })
    ).reset_index()

    W7.append(w7)

    print(w7)

      SEC AGE_GR    STRMID SSS   NS  nd_fsu         myfym           vyh  \
0       Z     ZZ  01010101   1  2.0       2  0.000000e+00  0.000000e+00   
1       Z     ZZ  01010101   2  2.0       1  2.610635e+03  2.047189e+04   
2       Z     ZZ  01010101   3  2.0       2  0.000000e+00  0.000000e+00   
3       Z     ZZ  01010101   4  2.0       2  0.000000e+00  0.000000e+00   
4       Z     ZZ  01010101   6  2.0       1  3.450790e+03  1.279493e+03   
...    ..    ...       ...  ..  ...     ...           ...           ...   
25723   Z     ZZ  02363222   7  2.0       2  0.000000e+00  0.000000e+00   
25724   Z     ZZ  02363223   2  2.0       1  2.858902e+06  7.236100e+04   
25725   Z     ZZ  02363223   4  2.0       1  2.991649e+04  6.512490e+05   
25726   Z     ZZ  02363223   6  2.0       1  1.657597e+06  8.755681e+06   
25727   Z     ZZ  02363223   7  2.0       2  0.000000e+00  0.000000e+00   

              myhyp  
0      0.000000e+00  
1      3.482372e+03  
2      0.000000e+00  
3      0.00

In [29]:
for i in range(len(W6)):

    ### Adding both the variances
    condition = (
        (W6[i]['SEC'].astype(str) + W6[i]['AGE_GR'].astype(str) + W6[i]['STRMID'].astype(str) + W6[i]['SSS'].astype(str)) ==
        (W7[i]['SEC'].astype(str) + W7[i]['AGE_GR'].astype(str) + W7[i]['STRMID'].astype(str) + W7[i]['SSS'].astype(str))
    )

    # Update 'w6' based on the condition
    W6[i].loc[condition, 'myfym'] += W7[i].loc[condition, 'myfym']
    W6[i].loc[condition, 'vyh'] += W7[i].loc[condition, 'vyh']
    W6[i].loc[condition, 'myhyp'] += W7[i].loc[condition, 'myhyp']

    ### Masking if the NS > 1
    mask = W6[i]['NS'] > 1

    # Apply the calculation using vectorized operations
    W6[i].loc[mask, 'vyh'] /= W6[i].loc[mask, 'NS'] * W6[i].loc[mask, 'NS'] - 1
    W6[i].loc[mask, 'myfym'] /= W6[i].loc[mask, 'NS'] * W6[i].loc[mask, 'NS'] - 1
    W6[i].loc[mask, 'myhyp'] /= W6[i].loc[mask, 'NS'] * W6[i].loc[mask, 'NS'] - 1

    # Generating Variance at Group level
    w8 = W6[i].groupby(['SEC', 'AGE_GR']).agg(
        vyh=('vyh', 'sum'),
        myfym=('myfym', 'sum'),
        myhyp=('myhyp', 'sum')
    ).reset_index()

    # Post related estimates from W1
    w9 = pd.merge(w8, W1[i], on=['SEC', 'AGE_GR'])

    T1 = w9.copy()
    T1['R1Hat'] = 1000 * T1['r1hat']
    T1['RSE_YfYm'] = np.where(T1['yfhat'] > 0, 100 * (T1['myfym'] ** 0.5) / T1['yfhat'], 0.00)
    T1['RSE_YH'] = np.where(T1['yhhat'] > 0, 100 * (T1['vyh'] ** 0.5) / T1['yhhat'], 0.00)
    T1['R2Hat'] = 100 * T1['r2hat']
    T1['RSE_YhYp'] = np.where(T1['yhhat'] > 0, 100 * (T1['myhyp'] ** 0.5) / T1['yhhat'], 0.00)

    T1['sec_desc'] = ''
    T1['agegr_desc'] = ''

    T1 = T1[['SEC', 'AGE_GR', 'sec_desc', 'no_sam', 'agegr_desc', 'R1Hat', 'RSE_YfYm', 'RSE_YH', 'R2Hat', 'RSE_YhYp']]

    print(T1)

    Tables.append(T1)


### Merging all tables
### ------------------
def merge_dataframes(tables):
    merged_df = tables[0]
    for df in tables[1:]:
        merged_df = pd.concat([merged_df, df], axis=0)
    return merged_df


df_Tables = merge_dataframes(Tables)

df_Tables

  SEC AGE_GR sec_desc  no_sam agegr_desc       R1Hat  RSE_YfYm    RSE_YH  \
0   Z     ZZ           576569             928.597649  0.447843  0.965784   

       R2Hat  RSE_YhYp  
0  18.078832  0.894489  
  SEC  AGE_GR sec_desc  no_sam agegr_desc       R1Hat  RSE_YfYm    RSE_YH  \
0   Z       1            37029             943.267497  2.361020  0.000000   
1   Z       2            48948             876.396872  2.109158  0.000000   
2   Z       3           110160             818.641864  1.434122  2.294414   
3   Z       4           302053             972.788591  0.446495  1.034285   
4   Z       5            78379             976.123799  1.483019  3.798704   

       R2Hat  RSE_YhYp  
0   0.000000  0.000000  
1   0.000000  0.000000  
2  11.217498  2.181537  
3  27.449417  0.882722  
4   9.254212  3.594445  
   SEC AGE_GR sec_desc  no_sam agegr_desc       R1Hat  RSE_YfYm    RSE_YH  \
0    1      Z           402589             937.957644  0.532629  1.384211   
1    2      Z           173980

,SEC,AGE_GR,sec_desc,no_sam,agegr_desc,R1Hat,RSE_YfYm,RSE_YH,R2Hat,RSE_YhYp
0,Z,ZZ,,576569,,928.597649,0.447843,0.965784,18.078832,0.894489
0,Z,1,,37029,,943.267497,2.361020,0.000000,0.000000,0.000000
1,Z,2,,48948,,876.396872,2.109158,0.000000,0.000000,0.000000
2,Z,3,,110160,,818.641864,1.434122,2.294414,11.217498,2.181537
3,Z,4,,302053,,972.788591,0.446495,1.034285,27.449417,0.882722
4,Z,5,,78379,,976.123799,1.483019,3.798704,9.254212,3.594445
0,1,Z,,402589,,937.957644,0.532629,1.384211,12.342941,1.320068
1,2,Z,,173980,,907.507687,0.827342,1.347471,31.209264,1.227012
0,1,1,,26930,,980.114192,2.830449,0.000000,0.000000,0.000000
1,1,2,,36049,,893.316788,2.516579,0.000000,0.000000,0.000000
